In [1]:
import time

import numpy as np
import pandas as pd
import torch
from torch import nn
from tqdm import tqdm

# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [2]:
# turn off warning
import logging
logging.getLogger('pyomo.core').setLevel(logging.ERROR)

## Problem Setting

In [3]:
# init
steepness = 50    # steepness factor
num_blocks = 100  # number of expression blocks
num_data = 9100   # number of data
test_size = 100   # number of test size
val_size = 1000   # number of validation size
train_size = num_data - test_size - val_size

In [4]:
# parameters as input data
p_low, p_high = 1.0, 8.0
a_low, a_high = 0.5, 4.5
p_train = np.random.uniform(p_low, p_high, (train_size, 1)).astype(np.float32)
p_test  = np.random.uniform(p_low, p_high, (test_size, 1)).astype(np.float32)
p_dev   = np.random.uniform(p_low, p_high, (val_size, 1)).astype(np.float32)
a_train = np.random.uniform(a_low, a_high, (train_size, num_blocks)).astype(np.float32)
a_test  = np.random.uniform(a_low, a_high, (test_size, num_blocks)).astype(np.float32)
a_dev   = np.random.uniform(a_low, a_high, (val_size, num_blocks)).astype(np.float32)

In [5]:
# nm datasets
from neuromancer.dataset import DictDataset
data_train = DictDataset({"p":p_train, "a":a_train}, name="train")
data_test = DictDataset({"p":p_test, "a":a_test}, name="test")
data_dev = DictDataset({"p":p_dev, "a":a_dev}, name="dev")
# torch dataloaders
from torch.utils.data import DataLoader
batch_size = 64
loader_train = DataLoader(data_train, batch_size, num_workers=0, collate_fn=data_train.collate_fn, shuffle=True)
loader_test = DataLoader(data_test, batch_size, num_workers=0, collate_fn=data_test.collate_fn, shuffle=False)
loader_dev = DataLoader(data_dev, batch_size, num_workers=0, collate_fn=data_dev.collate_fn, shuffle=True)

## Exact Solver

In [6]:
from src.problem import msRosenbrock
model = msRosenbrock(steepness, num_blocks, timelimit=60)

In [7]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for p, a in tqdm(list(zip(p_test, a_test))):
    # set params
    model.set_param_val({"p":p, "a":a})
    # solve
    tick = time.time()
    try:
        xval, objval = model.solve("scip")
        # eval
        params.append(list(p)+list(a))
        sols.append(list(list(xval.values())[0].values()))
        objvals.append(objval)
        conviols.append(sum(model.cal_violation()))
    except:
        params.append(list(p)+list(a))
        sols.append(None)
        objvals.append(None)
        conviols.append(None)
    tock = time.time()
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
df.to_csv("result/rb_exact_50-100.csv")
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
print("Number of None values: ", df["Sol"].isna().sum())

 18%|██████████████▏                                                                | 18/100 [18:01<1:22:05, 60.07s/it]

ERROR: Solver (scip) returned non-zero return code (4294967295)
ERROR: Solver log: SCIP version 8.1.0 [precision: 8 byte] [memory: block]
[mode: optimized] [LP solver: Soplex 6.0.4] [GitHash: 6129793871] Copyright
(c) 2002-2023 Zuse Institute Berlin (ZIB)

    External libraries:
      Soplex 6.0.4         Linear Programming Solver developed at Zuse
      Institute Berlin (soplex.zib.de) [GitHash: 950b1658] CppAD 20180000.0
      Algorithmic Differentiation of C++ algorithms developed by B. Bell
      (github.com/coin-or/CppAD) ZLIB 1.2.13          General purpose
      compression library by J. Gailly and M. Adler (zlib.net) AMPL/MP
      4e2d45c4     AMPL .nl file reader library (github.com/ampl/mp) PaPILO
      2.1.4         parallel presolve for integer and linear optimization
      (github.com/scipopt/papilo) [GitHash: ee0677c4] bliss 0.77
      Computing Graph Automorphism Groups by T. Junttila and P. Kaski
      (https://users.aalto.fi/~tjunttil/bliss/) Ipopt 3.14.13        Inte

100%|██████████████████████████████████████████████████████████████████████████████| 100/100 [1:39:12<00:00, 59.53s/it]


            Obj Val  Constraints Viol  Elapsed Time
count  9.900000e+01              99.0    100.000000
mean   8.961408e+05               0.0     59.528103
std    5.434359e+06               0.0      5.008952
min    6.193385e+02               0.0      9.979395
25%    7.900863e+02               0.0     59.954186
50%    8.886854e+02               0.0     60.008269
75%    1.032539e+03               0.0     60.050663
max    4.288076e+07               0.0     60.834042
Number of infeasible solution: 0
Number of None values:  1


## Heuristic - Round

In [8]:
from src.heuristic import naive_round

In [9]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for p, a in tqdm(list(zip(p_test, a_test))):
    # set params
    model.set_param_val({"p":p, "a":a})
    # relax
    model_rel = model.relax()
    # solve
    tick = time.time()
    xval_rel, _ = model_rel.solve("scip")
    xval, objval = naive_round(xval_rel, model)
    tock = time.time()
    # eval
    params.append(list(p)+list(a))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
df.to_csv("result/rb_heur_rnd_50-100.csv")
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [05:22<00:00,  3.22s/it]


            Obj Val  Constraints Viol  Elapsed Time
count  1.000000e+02        100.000000    100.000000
mean   3.290791e+05          4.781938      3.187064
std    2.595695e+06         26.067613      3.802257
min    3.449771e+02          0.000000      0.919436
25%    4.358581e+02          0.000000      1.688906
50%    5.005482e+02          0.000000      2.341156
75%    6.162474e+02          1.713147      3.439732
max    2.504485e+07        224.792820     37.490137
Number of infeasible solution: 36


## Heuristic - N1

In [10]:
model_heur = model.first_solution_heuristic(nodes_limit=1)

In [11]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for p, a in tqdm(list(zip(p_test, a_test))):
    # set params
    model_heur.set_param_val({"p":p, "a":a})
    # solve
    tick = time.time()
    try:
        xval, objval = model_heur.solve("scip")
        # eval
        params.append(list(p)+list(a))
        sols.append(list(list(xval.values())[0].values()))
        objvals.append(objval)
        conviols.append(sum(model_heur.cal_violation()))
    except:
        params.append(list(p)+list(a))
        sols.append(None)
        objvals.append(None)
        conviols.append(None)
    tock = time.time()
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
df.to_csv("result/rb_heur_n1_50-100.csv")
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:39<00:00,  2.52it/s]


            Obj Val  Constraints Viol  Elapsed Time
count  1.000000e+02             100.0    100.000000
mean   2.985608e+08               0.0      0.394850
std    1.402488e+09               0.0      0.160393
min    6.967802e+02               0.0      0.249860
25%    8.342075e+02               0.0      0.307133
50%    9.611967e+02               0.0      0.338061
75%    1.085219e+03               0.0      0.404864
max    8.248721e+09               0.0      1.135251
Number of infeasible solution: 0


## Learnable Rounding

In [7]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [8]:
# hyperparameters
penalty_weight = 100   # weight of constraint violation penealty
hlayers_sol = 5       # number of hidden layers for solution mapping
hlayers_rnd = 4       # number of hidden layers for solution mapping
hsize = 64            # width of hidden layers for solution mapping
lr = 1e-3             # learning rate

In [9]:
# set problem
import neuromancer as nm
from src.problem import nmRosenbrock
from src.func.layer import netFC
from src.func import roundGumbelModel
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_blocks+1, outsize=2*num_blocks, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["p", "a"], ["x"], name="smap")
# define rounding model
layers_rnd = netFC(input_dim=3*num_blocks+1, hidden_dims=[hsize]*hlayers_rnd, output_dim=2*num_blocks)
rnd = roundGumbelModel(layers=layers_rnd, param_keys=["p", "a"], var_keys=["x"],  output_keys=["x_rnd"], 
                       int_ind=model.int_ind, continuous_update=True, name="round")
# build neuromancer problem for rounding
components = nn.ModuleList([smap, rnd]).to("cuda")
loss_fn = nmRosenbrock(["p", "a", "x_rnd"], steepness, num_blocks, penalty_weight)

In [10]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs, patience, warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Validation Loss: 26030.72
Epoch 1, Validation Loss: 2930.95
Epoch 2, Validation Loss: 1616.89
Epoch 3, Validation Loss: 1341.72
Epoch 4, Validation Loss: 1513.75
Epoch 5, Validation Loss: 1351.38
Epoch 6, Validation Loss: 1042.79
Epoch 7, Validation Loss: 924.95
Epoch 8, Validation Loss: 964.55
Epoch 9, Validation Loss: 845.01
Epoch 10, Validation Loss: 783.51
Epoch 11, Validation Loss: 762.00
Epoch 12, Validation Loss: 783.09
Epoch 13, Validation Loss: 611.56
Epoch 14, Validation Loss: 642.49
Epoch 15, Validation Loss: 597.33
Epoch 16, Validation Loss: 622.12
Epoch 17, Validation Loss: 617.11
Epoch 18, Validation Loss: 614.68
Epoch 19, Validation Loss: 609.82
Epoch 20, Validation Loss: 619.65
Epoch 21, Validation Loss: 753.51
Epoch 22, Validation Loss: 674.03
Epoch 23, Validation Loss: 679.90
Epoch 24, Validation Loss: 585.54
Epoch 25, Validation Loss: 638.27
Epoch 26, Validation Loss: 539.36
Epoch 27, Validation Loss: 832.26
Epoch 28, Validation Loss: 870.48
Epoch 29, Valida

In [11]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for p, a in tqdm(list(zip(p_test, a_test))):
    # data point as tensor
    datapoints = {"p": torch.tensor(np.array([p]), dtype=torch.float32).to("cuda"), 
                  "a": torch.tensor(np.array([a]), dtype=torch.float32).to("cuda"),
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"p":p, "a":a})
    # assign vars
    x = datapoints["x_rnd"]
    for i in range(2*num_blocks):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval, objval = model.get_val()    
    params.append(list(p)+list(a))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
df.to_csv("result/rb_lr_50-100.csv")
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:01<00:00, 53.86it/s]


           Obj Val  Constraints Viol  Elapsed Time
count   100.000000        100.000000    100.000000
mean    505.468789          0.049710      0.003488
std     277.830401          0.497105      0.001259
min     205.342949          0.000000      0.001997
25%     275.317606          0.000000      0.002511
50%     439.786604          0.000000      0.003239
75%     649.349116          0.000000      0.004028
max    1594.439685          4.971045      0.006700
Number of infeasible solution: 1


## Learnable Threshold

In [12]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [13]:
# hyperparameters
penalty_weight = 100  # weight of constraint violation penealty
hlayers_sol = 5       # number of hidden layers for solution mapping
hlayers_rnd = 4       # number of hidden layers for solution mapping
hsize = 64            # width of hidden layers for solution mapping
lr = 1e-3             # learning rate

In [14]:
# set problem
import neuromancer as nm
from src.problem import nmRosenbrock
from src.func.layer import netFC
from src.func import roundThresholdModel
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_blocks+1, outsize=2*num_blocks, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["p", "a"], ["x"], name="smap")
# define rounding model
layers_rnd = netFC(input_dim=3*num_blocks+1, hidden_dims=[hsize]*hlayers_rnd, output_dim=2*num_blocks)
rnd = roundThresholdModel(layers=layers_rnd, param_keys=["p", "a"], var_keys=["x"],  output_keys=["x_rnd"], 
                          int_ind=model.int_ind, continuous_update=True, name="round")
# build neuromancer problem for rounding
components = nn.ModuleList([smap, rnd]).to("cuda")
loss_fn = nmRosenbrock(["p", "a", "x_rnd"], steepness, num_blocks, penalty_weight)

In [15]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs, patience, warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Validation Loss: 22853.77
Epoch 1, Validation Loss: 2533.40
Epoch 2, Validation Loss: 1181.49
Epoch 3, Validation Loss: 1128.61
Epoch 4, Validation Loss: 1078.11
Epoch 5, Validation Loss: 884.36
Epoch 6, Validation Loss: 805.50
Epoch 7, Validation Loss: 800.20
Epoch 8, Validation Loss: 782.83
Epoch 9, Validation Loss: 747.49
Epoch 10, Validation Loss: 628.15
Epoch 11, Validation Loss: 631.07
Epoch 12, Validation Loss: 599.40
Epoch 13, Validation Loss: 622.94
Epoch 14, Validation Loss: 661.64
Epoch 15, Validation Loss: 618.15
Epoch 16, Validation Loss: 680.72
Epoch 17, Validation Loss: 980.28
Epoch 18, Validation Loss: 625.20
Epoch 19, Validation Loss: 654.67
Epoch 20, Validation Loss: 590.01
Epoch 21, Validation Loss: 1621.59
Epoch 22, Validation Loss: 632.10
Epoch 23, Validation Loss: 696.41
Epoch 24, Validation Loss: 735.97
Epoch 25, Validation Loss: 640.66
Epoch 26, Validation Loss: 693.05
Epoch 27, Validation Loss: 652.99
Epoch 28, Validation Loss: 755.47
Epoch 29, Validat

In [16]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for p, a in tqdm(list(zip(p_test, a_test))):
    # data point as tensor
    datapoints = {"p": torch.tensor(np.array([p]), dtype=torch.float32).to("cuda"), 
                  "a": torch.tensor(np.array([a]), dtype=torch.float32).to("cuda"),
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"p":p, "a":a})
    # assign vars
    x = datapoints["x_rnd"]
    for i in range(2*num_blocks):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval, objval = model.get_val()    
    params.append(list(p)+list(a))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
df.to_csv("result/rb_lr_50-100.csv")
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:01<00:00, 53.97it/s]


          Obj Val  Constraints Viol  Elapsed Time
count  100.000000             100.0    100.000000
mean   608.469578               0.0      0.003483
std    107.705719               0.0      0.001365
min    371.036058               0.0      0.001351
25%    555.927263               0.0      0.002958
50%    600.248533               0.0      0.003018
75%    651.470496               0.0      0.003631
max    957.654814               0.0      0.010641
Number of infeasible solution: 0


## Parametric Learning Then Rounding

In [7]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [8]:
# hyperparameters
penalty_weight = 100  # weight of constraint violation penealty
hlayers_sol = 5       # number of hidden layers for solution mapping
hsize = 64            # width of hidden layers for solution mapping
lr = 1e-3             # learning rate
batch_size = 64       # batch size

In [10]:
# set problem
import neuromancer as nm
from src.problem import nmRosenbrock
from src.func.layer import netFC
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_blocks+1, outsize=2*num_blocks, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["p", "a"], ["x"], name="smap")
# build neuromancer problem for rounding
components = nn.ModuleList([smap]).to("cuda")
loss_fn = nmRosenbrock(["p", "a", "x"], steepness, num_blocks, penalty_weight)

In [11]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs, patience, warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Validation Loss: 22571.48
Epoch 1, Validation Loss: 2525.01
Epoch 2, Validation Loss: 915.34
Epoch 3, Validation Loss: 575.07
Epoch 4, Validation Loss: 413.19
Epoch 5, Validation Loss: 315.38
Epoch 6, Validation Loss: 338.62
Epoch 7, Validation Loss: 340.80
Epoch 8, Validation Loss: 290.29
Epoch 9, Validation Loss: 314.05
Epoch 10, Validation Loss: 364.39
Epoch 11, Validation Loss: 275.40
Epoch 12, Validation Loss: 260.99
Epoch 13, Validation Loss: 264.58
Epoch 14, Validation Loss: 288.60
Epoch 15, Validation Loss: 281.13
Epoch 16, Validation Loss: 282.78
Epoch 17, Validation Loss: 268.49
Epoch 18, Validation Loss: 244.98
Epoch 19, Validation Loss: 250.20
Epoch 20, Validation Loss: 289.79
Epoch 21, Validation Loss: 246.91
Epoch 22, Validation Loss: 290.32
Epoch 23, Validation Loss: 289.54
Epoch 24, Validation Loss: 271.49
Epoch 25, Validation Loss: 247.32
Epoch 26, Validation Loss: 222.55
Epoch 27, Validation Loss: 225.41
Epoch 28, Validation Loss: 288.51
Epoch 29, Validation 

In [12]:
from src.heuristic import naive_round
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for p, a in tqdm(list(zip(p_test, a_test))):
    # data point as tensor
    datapoints = {"p": torch.tensor(np.array([p]), dtype=torch.float32).to("cuda"), 
                  "a": torch.tensor(np.array([a]), dtype=torch.float32).to("cuda"),
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"p":p, "a":a})
    # assign vars
    x = datapoints["x"]
    for i in range(num_blocks*2):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval_rel, _ = model.get_val()
    xval, objval = naive_round(xval_rel, model)
    params.append(list(p)+list(a))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
df.to_csv("result/rb_pr_50-100.csv")
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:02<00:00, 42.26it/s]


          Obj Val  Constraints Viol  Elapsed Time
count  100.000000        100.000000    100.000000
mean   616.062211          1.176163      0.000890
std     81.610552          1.486732      0.000521
min    463.108093          0.000000      0.000000
25%    566.122963          0.000000      0.000998
50%    616.914702          0.281389      0.001001
75%    657.642093          2.374390      0.001033
max    858.509379          4.795009      0.002762
Number of infeasible solution: 55
